In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats
from scipy.stats import norm

In [2]:
years = 2
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365 * years)

In [3]:
tickers = ['TTWO', 'PLTR', 'PSKY', 'AAPL', 'NVDA', 'SHOP', 'BLK', 'JPM', 'HOOD', 'T', 'WMT', 'RL']

In [4]:
close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start = startDate, end =endDate)
    close_df[ticker] = data['Close']

print(close_df)

C:\Users\User\AppData\Local\Temp\ipykernel_11112\1321847875.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start = startDate, end =endDate)
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_11112\1321847875.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start = startDate, end =endDate)
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_11112\1321847875.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start = startDate, end =endDate)
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_11112\1321847875.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, st

                  TTWO        PLTR       PSKY        AAPL        NVDA  \
Date                                                                    
2023-11-06  134.369995   18.540001  12.280824  177.491486   45.724052   
2023-11-07  136.360001   18.799999  12.425988  180.056366   45.927933   
2023-11-08  143.470001   18.490000  11.438877  181.115967   46.546574   
2023-11-09  146.320007   18.270000  11.738883  180.640640   46.922344   
2023-11-10  147.880005   19.670000  11.709850  184.835114   48.306526   
...                ...         ...        ...         ...         ...   
2025-10-27  254.270004  189.179993  16.570000  268.809998  191.490005   
2025-10-28  251.570007  189.600006  16.209999  269.000000  201.029999   
2025-10-29  251.729996  198.809998  15.880000  269.700012  207.039993   
2025-10-30  252.779999  194.550003  15.460000  271.399994  202.889999   
2025-10-31  256.369995  200.470001  15.390000  270.369995  202.490005   

                  SHOP          BLK         JPM   

In [5]:
log_returns = np.log(close_df/close_df.shift(1))
log_returns = log_returns.dropna()
print(log_returns)

                TTWO      PLTR      PSKY      AAPL      NVDA      SHOP  \
Date                                                                     
2023-11-07  0.014701  0.013926  0.011751  0.014347  0.004449  0.020636   
2023-11-08  0.050828 -0.016627 -0.082772  0.005868  0.013380  0.016877   
2023-11-09  0.019670 -0.011970  0.025889 -0.002628  0.008041 -0.038507   
2023-11-10  0.010605  0.073834 -0.002476  0.022955  0.029073  0.015940   
2023-11-13  0.014834  0.002031 -0.008299 -0.008621  0.005879  0.016171   
...              ...       ...       ...       ...       ...       ...   
2025-10-27 -0.003337  0.024345 -0.009012  0.022535  0.027692  0.012126   
2025-10-28 -0.010675  0.002218 -0.021966  0.000707  0.048619  0.022034   
2025-10-29  0.000636  0.047433 -0.020568  0.002599  0.029458  0.000279   
2025-10-30  0.004162 -0.021660 -0.026804  0.006283 -0.020248 -0.030630   
2025-10-31  0.014102  0.029975 -0.004538 -0.003802 -0.001973  0.001439   

                 BLK       JPM      H

# Monte Carlo Simulation Method

In [6]:
def expected_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)

In [7]:
def standard_deviation (weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

In [8]:
cov_matrix = log_returns.cov()

In [9]:
portfolio_value = 1000000
weights = np.array([1/len(tickers)]*len(tickers))
portfolio_expected_return = expected_return(weights, log_returns)
portfolio_std_dev = standard_deviation(weights, cov_matrix)

In [10]:
def random_z_score():
    return np.random.normal(0,1)

In [11]:
days = 5

def scenario_gain_loss(portfolio_value, portfolio_std_dev, z_score, days):
    return portfolio_value * (portfolio_expected_return * days + portfolio_std_dev * z_score * np.sqrt(days))

In [12]:
simulations = 10000
scenarionReturn = []

for i in range(simulations):
    z_score = random_z_score()
    scenarionReturn.append(scenario_gain_loss(portfolio_value, portfolio_std_dev, z_score, days))

In [27]:
confidence_levels = [0.9, 0.95, 0.99]
VaRs = []

for cl in confidence_levels:
    VaR = -np.percentile(scenarionReturn, 100*(1-cl))
    VaRs.append(VaR)

print (f'{"confidence Level":<20} {"Value at Risk":<20}')
print('.' * 40)
for cl, VaR in zip(confidence_levels, VaRs):
    print (f'{cl * 100:>6.0f}%: {"":<8} ${VaR:>10,.2f}')

confidence Level     Value at Risk       
........................................
    90%:          $ 31,255.76
    95%:          $ 42,751.15
    99%:          $ 64,398.81


# Historical Method

In [14]:
historical_returns = (log_returns * weights).sum(axis=1)

In [15]:
days = 5 

range_returns = historical_returns.rolling(window = days).sum()
range_returns = range_returns.dropna()

In [22]:
confidence_levels = [0.9, 0.95, 0.99]
VaRs = []

for cl in confidence_levels:
    VaR = -np.percentile(range_returns, 100*(1-cl))*portfolio_value
    VaRs.append(VaR)

print (f'{"confidence Level":<20} {"Value at Risk":<20}')
print('.' * 40)
for cl, VaR in zip(confidence_levels, VaRs):
    print (f'{cl * 100:>6.0f}%: {"":<8} ${VaR:>10,.2f}')

confidence Level     Value at Risk       
........................................
    90%:          $ 27,454.57
    95%:          $ 41,419.71
    99%:          $ 84,273.45


# Parametric Method

In [17]:
cov_matrix = log_returns.cov () * 252

In [18]:
confidence_levels = [0.9, 0.95, 0.99]

In [23]:
VaRs = []

for cl in confidence_levels:
    VaR = portfolio_value * portfolio_std_dev * norm.ppf(cl) * np.sqrt(days)
    VaRs.append(VaR)


print (f'{"confidence Level":<20} {"Value at Risk":<20}')
print('.' * 40)
for cl, VaR in zip(confidence_levels, VaRs):
    print (f'{cl * 100:>6.0f}%: {"":<8} ${VaR:>10,.2f}')

confidence Level     Value at Risk       
........................................
    90%:          $ 41,485.08
    95%:          $ 53,245.52
    99%:          $ 75,306.16
